<a href="https://colab.research.google.com/github/Strincience/CTC-Audio-Transcription/blob/main/Audio_to_Text_Transcriber_(Offline).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchaudio transformers librosa

In [ ]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa
import os

In [ ]:
# # Loading the processor and the model
# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
# model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# # Sending the model to te device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)


# Initialize processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

In [ ]:
# ## Defining the folder path
# # import os

# folder_path = r"/content/drive/MyDrive/LibriSpeech/dev-clean/1272"

# # for file in os.listdir(folder_path):
# #     if file.endswith(".flac") or filee.endswith(".wav")
# #         full_path = os.path.join(folder_path, file)
# #         print(f"🔊 Transcribing {file}...")
# #         print("📜", transcribe_audio(full_path))
# #         print("-" * 40)



In [ ]:
def load_audio(path):

  # Loading the audio file
  waveform, sample_rate = torchaudio.load(path)
  waveform = waveform.to(device)

  # Resampling to 16000 kHz if needed
  if sample_rate != 16000:
    print("⚠️ The sample Rate {sample_rate} is not 16000, resampling...")
    resampler = torchaudio.transforms.Resample(sample_rate, 16000).to(device)
    waveform = resampler(waveform)
  return waveform.squeeze()
  # input_values = processor(waveform,
  #                          sampling_rate=sample_rate,
  #                          return_tensors="pt",
  #                          padding="longest").input_values.to(device)



In [ ]:
def transcribe(audio_path, processor, model):
  try:
    # load audio
    waveform = load_audio(audio_path)
    # Transcribing the audio
    input_values = processor(waveform,
                             sampling_rate=16000,
                             return_tensors="pt",
                             padding="longest").input_values.to(device)
    # Get Transcription
    with torch.no_grad():
      logits = model(input_values).logits
      # Giving each audio an ID.
      predicted_ids  = torch.argmax(logits, dim = -1)
      # Transcribing the audio into human-readable text.
      transcription = processor.decode(predicted_ids[0])

    return transcription

  except Exception as e:
    print(f"Error Processing {audio_path} : {str(e)}")
    return None





  # ## Code saninty check

  # print("🚀 Starting transcription process...")
  # print("🔍 Looking for audio files in:", folder_path)

  # if len(os.listdir(folder_path)) == 0:
  #   print("❌ No files found in the folder.")
  # else:
  #   print("📂 Files in folder:", os.listdir(folder_path))


  # ## Printing the results
  # print(f" 🎧 {os.path.basename(full_path)}")
  # print(" 📜", transcription)
  # print("-"* 60)


if __name__ == "__main__":
  folder_path = r"/content/drive/MyDrive/LibriSpeech/Audio Files"
  if not os.path.exists(folder_path):
    print(f"❌ Folder not found: {folder_path}")
  else:
    print(f"processing files in: {folder_path}")


  for file_name in os.listdir(folder_path):
    if file_name.endswith((".flac", ".wav")):
      audio_files = os.path.join(folder_path, file_name)
      print(f"\n🔊 Processing: {folder_path}")


      transcription = transcribe(audio_files, processor, model)
      print("📜", transcription)
      print("-" * 40)

      # transcribe(full_path, processor, model)

processing files in: /content/drive/MyDrive/LibriSpeech/Audio Files

🔊 Processing: /content/drive/MyDrive/LibriSpeech/Audio Files
📜 FOR SOME TIME NOTHING WAS HEARD IN THAT CHAMBER BUT SOBS EXCLAMATIONS AND PRAYERS
----------------------------------------

🔊 Processing: /content/drive/MyDrive/LibriSpeech/Audio Files
📜 BUT IN LESS THAN FIVE MINUTES THE STAIRCASE GROANED BENEATH AN EXTRAORDINARY WEIGHT
----------------------------------------

🔊 Processing: /content/drive/MyDrive/LibriSpeech/Audio Files
📜 DO YOU KNOW THE ASSASSIN ASKED MORREL
----------------------------------------

🔊 Processing: /content/drive/MyDrive/LibriSpeech/Audio Files
📜 AND THE CRY ISSUED FROM HIS PORS IF WE MAY THUS SPEAK A CRY FRIGHTFUL IN ITS SILENCE
----------------------------------------

🔊 Processing: /content/drive/MyDrive/LibriSpeech/Audio Files
📜 OH YOU RAVE SIR EXCLAIMED VILLEFORT IN VAIN ENDEAVORING TO ESCAPE THE NET IN WHICH HE WAS TAKEN I RAVE
----------------------------------------

🔊 Processing: 

# To confirm that there are flac files in the folder

In [ ]:
import os

audio_folder = "/content/drive/MyDrive/LibriSpeech/Audio Files"
audio_files = [f for f in os.listdir(audio_folder) if f.endswith((".wav", ".flac"))]

print(f"Found {len(audio_files)} audio files:")
print(audio_files[:5])  # Show a few

Found 28 audio files:
['Copy of 84-121123-0006.flac', 'Copy of 84-121123-0001.flac', 'Copy of 84-121123-0009.flac', 'Copy of 84-121123-0003.flac', 'Copy of 84-121123-0008.flac']
